In [ ]:
import pandas as pd
import numpy as np
import loglikelihood_utils as ll
import logit

This notebook demonstrates how to compare different models using the likelihood ratio test.
This test allows to check that the improvements in likelihood that come from a more complex specification
do not simply come from the additional degrees of freedom offered to the model.

# Estimate a First Model

In [ ]:
# Read and transform dataset in one go
dataset = pd.read_csv('swissmetro.dat', sep='\t')\
    .query('(PURPOSE == 1 or PURPOSE == 3) and CHOICE != 0')\
    .assign(car_av_sp = lambda df: ll.avail(df.CAR_AV * (df.SP != 0)),
            train_av_sp = lambda df: ll.avail(df.TRAIN_AV * (df.SP != 0)),
            sm_av = lambda df: ll.avail(df.SM_AV),
            train_cost = lambda df: df.TRAIN_CO * (df.GA == 0),
            sm_cost = lambda df: df.SM_CO * (df.GA == 0))
dataset.head()

In [ ]:
# This will be the initial starting point.
betas = ll.Betas(asc_car=0.,
                    asc_train=0.,
                    asc_sm=ll.Beta(0., fixed=True),
                    time=0.,
                    cost=0.)

utilities = {
    # train
    1: lambda b, d: betas.get('asc_train', b) +
                        betas.get('time', b) * d['TRAIN_TT'] / 100. +
                        betas.get('cost', b) * d['train_cost'] / 100. +
                        d['train_av_sp'],
    # SwissMetro
    2: lambda b, d: betas.get('asc_sm', b) +
                        betas.get('time', b) * d['SM_TT'] / 100. +
                        betas.get('cost', b) * d['sm_cost'] / 100. +
                        d['sm_av'],
    # Car
    3: lambda b, d: betas.get('asc_car', b) +
                        betas.get('time', b) * d['CAR_TT'] / 100. +
                        betas.get('cost', b) * d['CAR_CO'] / 100. +
                        d['car_av_sp']
}

In [ ]:
estimates = logit.estimate(betas, utilities, dataset.CHOICE, dataset)

In [ ]:
estimates

# Estimate a Second Model

Maybe adding a log term for time and cost works better?

We simply add terms to the utilities, so the previous model is a resticted version of the new one

In [ ]:
# This will be the initial starting point.
betas_ext = ll.Betas(asc_car=0.,
                    asc_train=0.,
                    asc_sm=ll.Beta(0., fixed=True),
                    time=0.,
                    cost=0.,
                    logtime=0.,
                    logcost=0.)


utilities_ext = {
    # train
    1: lambda b, d: betas_ext.get('asc_train', b) +
                        betas_ext.get('time', b) * d['TRAIN_TT'] / 100. +
                        betas_ext.get('cost', b) * d['train_cost'] / 100. +
                        betas_ext.get('logtime', b) * np.log(d['TRAIN_TT'] + 1) +
                        betas_ext.get('logcost', b) * np.log(d['train_cost'] + 1) +
                        d['train_av_sp'],
    # SwissMetro
    2: lambda b, d: betas_ext.get('asc_sm', b) +
                        betas_ext.get('time', b) * d['SM_TT'] / 100. +
                        betas_ext.get('cost', b) * d['sm_cost'] / 100. +
                        betas_ext.get('logtime', b) * np.log(d['SM_TT'] + 1) +
                        betas_ext.get('logcost', b) * np.log(d['sm_cost'] + 1) +
                        d['sm_av'],
    # Car
    3: lambda b, d: betas_ext.get('asc_car', b) +
                        betas_ext.get('time', b) * d['CAR_TT'] / 100. +
                        betas_ext.get('cost', b) * d['CAR_CO'] / 100. +
                        betas_ext.get('logtime', b) * np.log(d['CAR_TT'] + 1) +
                        betas_ext.get('logcost', b) * np.log(d['CAR_CO'] + 1) +
                        d['car_av_sp']
}

In [ ]:
estimates_ext = logit.estimate(betas_ext, utilities_ext, dataset.CHOICE, dataset)

In [ ]:
estimates_ext

In [ ]:
ll.likelihood_ratio_p_value(estimates, estimates_ext)

Model is indeed better

# Estimate a Third Model

Ok, so adding a log term is better than only linear terms. Do the linear terms actually bring something?

We remove the linear terms from the previous model, creating a restricted version.

Note that this new model **cannot** be compared to the initial model, as one cannot go from one of the models to the others by imposing constraints on parameters.

In [ ]:
# This will be the initial starting point.
betas_log = ll.Betas(asc_car=0.,
                    asc_train=0.,
                    asc_sm=ll.Beta(0., fixed=True),
                    logtime=0.,
                    logcost=0.)


utilities_log = {
    # train
    1: lambda b, d: betas_log.get('asc_train', b) +
                        betas_log.get('logtime', b) * np.log(d['TRAIN_TT'] + 1) +
                        betas_log.get('logcost', b) * np.log(d['train_cost'] + 1) +
                        d['train_av_sp'],
    # SwissMetro
    2: lambda b, d: betas_log.get('asc_sm', b) +
                        betas_log.get('logtime', b) * np.log(d['SM_TT'] + 1) +
                        betas_log.get('logcost', b) * np.log(d['sm_cost'] + 1) +
                        d['sm_av'],
    # Car
    3: lambda b, d: betas_log.get('asc_car', b) +
                        betas_log.get('logtime', b) * np.log(d['CAR_TT'] + 1) +
                        betas_log.get('logcost', b) * np.log(d['CAR_CO'] + 1) +
                        d['car_av_sp']
}

In [ ]:
estimates_log = logit.estimate(betas_log, utilities_log, dataset.CHOICE, dataset)

In [ ]:
estimates_log

In [ ]:
ll.likelihood_ratio_p_value(estimates_log, estimates_ext)

Yes, the linear terms also bring something

# Example of an Extended Model That Does Not Bring Improvement

Until now, the extended model was always better to a high degree of confidence.
This does not have to be the case (and this is the reason we do a test in the first place).
To exemplify this, we will now specify a model that cannot bring anything, by construction:
we will add dependence on a random variable, that we will name "phase of the moon".

In [ ]:
dataset_moon = dataset.assign(phase_moon=np.random.rand(dataset.shape[0]))

In [ ]:
# This will be the initial starting point.
betas_moon = ll.Betas(asc_car=0.,
                asc_train=0.,
                asc_sm=ll.Beta(0., fixed=True),
                time=0.,
                cost=0.,
                moon=0.)

utilities_moon = {
    # train
    1: lambda b, d: betas_moon.get('asc_train', b) +
                        betas_moon.get('time', b) * d['TRAIN_TT'] / 100. +
                        betas_moon.get('cost', b) * d['train_cost'] / 100. +
                        betas_moon.get('moon', b) * d['phase_moon'] +
                        d['train_av_sp'],
    # SwissMetro
    2: lambda b, d: betas_moon.get('asc_sm', b) +
                        betas_moon.get('time', b) * d['SM_TT'] / 100. +
                        betas_moon.get('cost', b) * d['sm_cost'] / 100. +
                        betas_moon.get('moon', b) * d['phase_moon'] +
                        d['sm_av'],
    # Car
    3: lambda b, d: betas_moon.get('asc_car', b) +
                        betas_moon.get('time', b) * d['CAR_TT'] / 100. +
                        betas_moon.get('cost', b) * d['CAR_CO'] / 100. +
                        betas_moon.get('moon', b) * d['phase_moon'] +
                        d['car_av_sp']
}

In [ ]:
estimates_moon = logit.estimate(betas_moon, utilities_moon, dataset_moon.CHOICE, dataset_moon)

In [ ]:
ll.likelihood_ratio_p_value(estimates, estimates_moon)

As expected, we can reject the fact that the model brings any improvement with really high confidence.